In [109]:
from pyspark.sql import SparkSession
import numpy as np
import itertools
import math

In [2]:
# create spark session
app_name = "final_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [5]:
# get 10000 data points for testing the gd 
!head -n 10000 dac/sample_train.txt > dac/sample_train_10000.txt

### Factorization machines using gradient descent

In [36]:
sample_rdd = sc.textFile('dac/sample_train_10000.txt').cache()

In [52]:
# initialize learning rate, number of latent factors, mean, standard deviation
learning_rate = 0.01
num_latent_factors = 10
# used for initializing the parameters for factorized interactions
mean = 0
# used for initializing the parameters for factorized interactions
std = 0.01

In [47]:
# initializing the regularization parameters
# techincally these should come from cross validation
reg_bias = 0.01
reg_independent = 0.01
reg_interaction = np.full(num_latent_factors, 0.01)

In [137]:
def prediction_calculation(features, w0, W, V):
    # dot product of independent features and weights
    # appending the bias term as well
    independent_features = np.append([1.0], features)
    independent_weights = np.append([w0], W)
    
    # get combinations of features for the interaction terms with degree 2
    total_interaction = 0.0
    for subset in itertools.combinations(enumerate(features),2):
        index1 = int(subset[0][0])
        index2 = int(subset[1][0])
        feature1 = float(subset[0][1])
        feature2 = float(subset[1][1])
        # dot product of v parameters and features
        total_interaction += np.dot(V[index1],V[index2]) * feature1 * feature2
    y_hat = np.dot(independent_features,independent_weights) + total_interaction
    return y_hat

In [138]:
def get_w0_gradients(data):
    y = data[0]
    features = data[1:]
    y_hat = prediction_calculation(features, w0_b.value, W_b.value, V_b.value)
    gradient = ((1.0/(1.0+math.exp(-y*y_hat))) - 1)*y
    yield gradient

In [139]:
def get_W_gradients(data):
    y = data[0]
    features = data[1:]
    y_hat = prediction_calculation(features, w0_b.value, W_b.value, V_b.value)
    gradient = ((1.0/(1.0+math.exp(-y*y_hat))) - 1)*y*features
    yield gradient

In [230]:
def get_V_gradients(data):
    y = data[0]
    features = data[1:]
    len_features = len(features)
    V = V_b.value
    y_hat = prediction_calculation(features, w0_b.value, W_b.value, V)
    # calculating the sum part of the the differenciation_interaction
    sum_v_features = np.empty((len_features,num_latent_factors_b.value))
    # executing xl*sum(vj,f * xj) where j!= l (for all j features except l)
    for i, feature in enumerate(features):
        # handle the first and last elements
        if i == 0:
            sum_v_features[i] = np.dot(features[i+1:],V[i+1:,:])
            continue
        if i == len_features-1:
            sum_v_features[i] = np.dot(features[:i],V[:i,:])
            continue
        sum_v_features[i] = np.dot(np.concatenate((features[:i], features[i+1:])),np.concatenate((V[:i,:], V[i+1:,:])))
    # reshape the feature vector to be able to multiple each feature with it's sum of (vj,f*xj) to compute the expression
    # xl*sum(vj,f * xj) where j!= l (for all j features except l)
    feature_reshape = np.reshape(features, (len_features, 1))
    # now multiply to get the entire V at once
    differenciation_interaction = np.multiply(feature_reshape,sum_v_features)
    gradient = ((1.0/(1.0+math.exp(-y*y_hat))) - 1)*y*differenciation_interaction
    yield gradient

In [242]:
def parse(line):
    data = np.array(line.split('\t'), dtype = 'float')
    yield data

In [247]:
def fm_gd(data_rdd, learning_rate, num_latent_factors, mean, std, reg_bias, reg_independent, reg_interaction, num_steps):    
    # -1 to not count the output variable
    num_features = len(data_rdd.take(1)[0].split('\t'))-1 
    
    # initialize the model parameters
    w0 = 0.0
    W = np.zeros(num_features)
    V = np.random.normal(mean, std, (num_features,num_latent_factors))
    
    for i in range(num_steps):
        # broadcast the model parameters
        w0_b = sc.broadcast(w0)
        W_b = sc.broadcast(W)
        V_b = sc.broadcast(V)
        num_latent_factors_b = sc.broadcast(num_latent_factors)
        
        # split and create array
        parse_data_rdd = data_rdd.flatMap(parse).cache() # map or flat map?
        
        # perform w0 updates
        w0_gradient = parse_data_rdd.flatMap(get_w0_gradients).mean()
        w0 -= learning_rate * w0_gradient + 2 * reg_bias.value * w0
        
        # perform W updates
        W_gradient = parse_data_rdd.flatMap(get_W_gradients).mean()
        W -= learning_rate * W_gradient + 2 * reg_independent.value * W
        
        # perform V updates
        V_gradient = parse_data_rdd.flatMap(get_V_gradients).mean()
        V -= learning_rate * V_gradient + 2 * np.multiply(reg_interaction.value, V)
        
    return w0, W, V

In [ ]:
w0, W, V = fm_gd(sample_rdd, learning_rate, num_latent_factors, mean, std, reg_bias, reg_independent, reg_interaction, 10)


In [ ]:
# TODO predict

### Testing the implementation

In [81]:
# making a dataset of just few rows and columns to test the implementation
test_rdd = sc.textFile('dac/sample_test_impl.txt').cache()

In [89]:
# initialize learning rate, number of latent factors, mean, standard deviation
learning_rate = 0.01
num_latent_factors = 2
# used for initializing the parameters for factorized interactions
mean = 0
# used for initializing the parameters for factorized interactions
std = 0.01
reg_bias = 0.01
reg_independent = 0.01
reg_interaction = np.full(num_latent_factors, 0.01)

In [90]:
num_features = len(test_rdd.take(1)[0].split('\t'))-1 
# initialize the model parameters
w0 = 0.0
W = np.zeros(num_features)
V = np.random.normal(mean, std, (num_features,num_latent_factors))

In [95]:
V

array([[-4.16864343e-03,  9.91140812e-03],
       [-1.47871127e-02, -5.82742997e-03],
       [-3.38995550e-03, -8.70194226e-05],
       [ 7.25278359e-04, -1.04087130e-02]])

In [102]:
x1 = np.dot(V[0],V[1])*1 + np.dot(V[0],V[2])*5 + np.dot(V[0],V[3])*0 + np.dot(V[1],V[2])*5 + np.dot(V[1],V[3])*0 + np.dot(V[2],V[3])*0
x2 = np.dot(V[0],V[1])*6 + np.dot(V[0],V[2])*8 + np.dot(V[0],V[3])*2 + np.dot(V[1],V[2])*12 + np.dot(V[1],V[3])*3 + np.dot(V[2],V[3])*4
x3 = np.dot(V[0],V[1])*40 + np.dot(V[0],V[2])*44 + np.dot(V[0],V[3])*4 + np.dot(V[1],V[2])*110 + np.dot(V[1],V[3])*10 + np.dot(V[2],V[3])*11
x4 = np.dot(V[0],V[1])*1 + np.dot(V[0],V[2])*5 + np.dot(V[0],V[3])*0 + np.dot(V[1],V[2])*5 + np.dot(V[1],V[3])*0 + np.dot(V[2],V[3])*0


In [110]:
gradient1= ((1.0/(1.0+math.exp(-0*x1))) - 1)*0
gradient2= ((1.0/(1.0+math.exp(-1*x2))) - 1)*1
gradient3= ((1.0/(1.0+math.exp(-0*x3))) - 1)*0
gradient4= ((1.0/(1.0+math.exp(-1*x4))) - 1)*1

In [111]:
[gradient1,gradient2,gradient3,gradient4]

[-0.0, -0.4998329300991783, -0.0, -0.49991914922926506]

In [113]:
sum([gradient1,gradient2,gradient3,gradient4])/4

-0.24993801983211084

In [125]:
total_interaction = 0.0
for subset in itertools.combinations(enumerate([1,1,5,0]),2):
    index1 = subset[0][0]
    index2 = subset[1][0]
    feature1 = subset[0][1]
    feature2 = subset[1][1]
    # dot product of v parameters and features
    total_interaction += np.dot(V[index1],V[index2]) * feature1 * feature2

In [126]:
total_interaction

0.0003234030857583696

In [182]:
# broadcast the model parameters
w0_b = sc.broadcast(w0)
W_b = sc.broadcast(W)
V_b = sc.broadcast(V)
num_latent_factors_b = sc.broadcast(num_latent_factors)

In [135]:
def get_features(line):
    data = line.split('\t')
    y = data[0]
    features = data[1:]
    yield features

In [145]:
w0_gradient = test_rdd.flatMap(parse).flatMap(get_w0_gradients).mean()
w0_gradient

-0.24993801983211084

In [146]:
gradient1w= ((1.0/(1.0+math.exp(-0*x1))) - 1)*0*np.array([1,1,5,0])
gradient2w= ((1.0/(1.0+math.exp(-1*x2))) - 1)*1*np.array([2,3,4,1])
gradient3w= ((1.0/(1.0+math.exp(-0*x3))) - 1)*0*np.array([4,10,11,1])
gradient4w= ((1.0/(1.0+math.exp(-1*x4))) - 1)*1*np.array([1,1,5,0])

In [149]:
sum([gradient1w,gradient2w,gradient3w,gradient4w])/4

array([-0.37489625, -0.49985448, -1.12473187, -0.12495823])

In [152]:
w_gradient = test_rdd.flatMap(parse).flatMap(get_W_gradients).mean()
w_gradient

array([-0.37489625, -0.49985448, -1.12473187, -0.12495823])

In [157]:
grad_1 = 2 * (np.multiply(V[1],3) + np.multiply(V[2],4) + np.multiply(V[3],1))
grad_2 = 3 * (np.multiply(V[0],2) + np.multiply(V[2],4) + np.multiply(V[3],1))
grad_3 = 4 * (np.multiply(V[1],3) + np.multiply(V[0],2) + np.multiply(V[3],1))
grad_4 = 1 * (np.multiply(V[1],3) + np.multiply(V[2],4) + np.multiply(V[0],2))

In [159]:
np.array([grad_1,grad_2,grad_3,grad_4])

array([[-0.11439176, -0.05647816],
       [-0.06351549,  0.02719808],
       [-0.20789339, -0.03227275],
       [-0.06625845,  0.00199245]])

In [211]:
gradient2V= ((1.0/(1.0+math.exp(-1*x2))) - 1)*1*np.array([grad_1,grad_2,grad_3,grad_4])

In [212]:
gradient2V

array([[ 0.05717677,  0.02822964],
       [ 0.03174713, -0.01359449],
       [ 0.10391196,  0.01613098],
       [ 0.03311815, -0.00099589]])

In [214]:
grad_1 = (np.multiply(V[1],3) + np.multiply(V[2],4) + np.multiply(V[3],1))
grad_2 = (np.multiply(V[0],2) + np.multiply(V[2],4) + np.multiply(V[3],1))
grad_3 = (np.multiply(V[1],3) + np.multiply(V[0],2) + np.multiply(V[3],1))
grad_4 = (np.multiply(V[1],3) + np.multiply(V[2],4) + np.multiply(V[0],2))

In [215]:
np.array([grad_1,grad_2,grad_3,grad_4])

array([[-0.05719588, -0.02823908],
       [-0.02117183,  0.00906603],
       [-0.05197335, -0.00806819],
       [-0.06625845,  0.00199245]])

In [227]:
np.concatenate((grad_1, grad_2))

array([-0.05719588, -0.02823908, -0.02117183,  0.00906603])

In [228]:
def get_V(data):
    y = data[0]
    features = data[1:]
    len_features = len(features)
    V = V_b.value
    y_hat = prediction_calculation(features, w0_b.value, W_b.value, V)
    # calculating the sum part of the the differenciation_interaction
    sum_v_features = np.zeros((len_features,num_latent_factors_b.value))
    for i, feature in enumerate(features):
        if i == 0:
            sum_v_features[i] = np.dot(features[i+1:],V[i+1:,:]) # check this :
            continue
        if i == len_features -1:
            sum_v_features[i] = np.dot(features[:i],V[:i,:]) # check this :
            continue
        sum_v_features[i] = np.dot(np.concatenate((features[:i], features[i+1:])),np.concatenate((V[:i,:], V[i+1:,:])))
    yield sum_v_features

In [231]:
v_gradient = test_rdd.flatMap(parse).flatMap(get_V_gradients).collect()
v_gradient

[array([[ 0.,  0.],
        [ 0., -0.],
        [ 0., -0.],
        [ 0., -0.]]), array([[ 0.05717677,  0.02822964],
        [ 0.03174713, -0.01359449],
        [ 0.10391196,  0.01613098],
        [ 0.03311815, -0.00099589]]), array([[ 0.,  0.],
        [ 0., -0.],
        [ 0.,  0.],
        [ 0.,  0.]]), array([[ 0.01586588,  0.00313076],
        [ 0.0105575 , -0.00473739],
        [ 0.04738173, -0.01020829],
        [ 0.        , -0.        ]])]

### Testing array manipulations

In [59]:
for subset in itertools.combinations(enumerate([1,2,3]),2):
    print(subset[0][0],subset[1][0],subset[0][1],subset[1][1])

0 1 1 2
0 2 1 3
1 2 2 3


In [64]:
np.dot([[1,2,3]],[[1,1,1],[1,0,1],[2,2,2]])

array([[9, 7, 9]])

In [70]:
np.multiply([[1],[2],[3]],[[1,1,1],[1,0,1],[2,2,2]])

array([[1, 1, 1],
       [2, 0, 2],
       [6, 6, 6]])

In [74]:
np.reshape([1,2,3], (3, 1))

array([[1],
       [2],
       [3]])

In [75]:
np.multiply(np.reshape([1,2,3], (3, 1)),[[1,1,1],[1,0,1],[2,2,2]])

array([[1, 1, 1],
       [2, 0, 2],
       [6, 6, 6]])

In [80]:
test = np.multiply([1,2,3],[[1,1,1],[1,0,1],[2,2,2]])
test

array([[1, 2, 3],
       [1, 0, 3],
       [2, 4, 6]])

In [79]:
test[0]

array([1, 2, 3])

In [84]:
np.multiply(2,[1,2,3])

array([2, 4, 6])

In [87]:
2.0 * test

array([[ 2.,  4.,  6.],
       [ 2.,  0.,  6.],
       [ 4.,  8., 12.]])

In [120]:
test[1:,:]

array([[1, 0, 3],
       [2, 4, 6]])

### Some experimenting with data frames

In [8]:
sample_df = spark.read.option('header', 'false').csv('dac/sample_train_10000.txt', sep='\t')

In [10]:
# save the data into parquet file
sample_df.write.format('parquet').save('dac/sample_train_10000.parquet')

In [11]:
sample_parquet_df = spark.read.parquet('dac/sample_train_10000.parquet')

In [29]:
sample_parquet_df.head(1)

[Row(_c0='0', _c1='1', _c2='1', _c3='5', _c4='0', _c5='1382', _c6='4', _c7='15', _c8='2', _c9='181', _c10='1', _c11='2', _c12=None, _c13='2', _c14='68fd1e64', _c15='80e26c9b', _c16='fb936136', _c17='7b4723c4', _c18='25c83c98', _c19='7e0ccccf', _c20='de7995b8', _c21='1f89b562', _c22='a73ee510', _c23='a8cd5504', _c24='b2cb9c98', _c25='37c9c164', _c26='2824a5f6', _c27='1adce6ef', _c28='8ba8b39a', _c29='891b62e7', _c30='e5ba7672', _c31='f54016b9', _c32='21ddcdc9', _c33='b1252a9d', _c34='07b5194c', _c35=None, _c36='3a171ecb', _c37='c5c50484', _c38='e8b83407', _c39='9727dd16')]

In [20]:
select_df = sample_parquet_df.select((sample_parquet_df._c1 + 10).alias('w0'))

In [28]:
sample_parquet_df.select(sample_parquet_df.columns[1:10]).head()

Row(_c1='1', _c2='1', _c3='5', _c4='0', _c5='1382', _c6='4', _c7='15', _c8='2', _c9='181')

In [ ]:
for i in range(10):
    sample_parquet_df.select((sample_parquet_df._c1 + 10).alias('w0'))